In [62]:
import pandas as pd
import re
import requests

In [63]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client.companies

In [64]:
df = db.companies

In [66]:
companies_with_coord = pd.DataFrame(df.find(
    {'$and': 
     [{'or':[
         {'category_code':'games_video'},
         {'category_code':'web'},
         {'category_code':'software'},
         {'category_code':'mobile'},
         {'category_code':'advertising'},
         {'category_code':'ecommerce'},
         {'category_code':'ecommerce'}
     ]},
         {"offices.latitude": 
          {'$exists': True}
         },
         {"offices.longitude": 
          {'$exists': True}
         },
         {"offices.latitude": 
          {'$not': 
           {'$eq': None}
          }
         },
         {"offices.longitude": 
          {'$not': 
           {'$eq': None}
          }
         },
         {'deadpooled_year': 
             {'$eq' :None}
         }
     ]
    }))

In [ ]:
{$and:[{"offices.latitude": {$exists: True}},{"offices.longitude": {$exists: True}},{"offices.latitude": {$not: {$eq: null}}},{"offices.longitude": {$not: {$eq: null}}},{'deadpooled_year': {$eq :None}}]}

In [68]:
display(companies_with_coord.head())

,_id,acquisition,acquisitions,alias_list,blog_feed_url,blog_url,category_code,competitions,created_at,crunchbase_url,...,phone_number,products,providerships,relationships,screenshots,tag_list,total_money_raised,twitter_username,updated_at,video_embeds
0,52cdef7c4bab8bd675297d8d,"{'price_amount': 500000, 'price_currency_code'...","[{'price_amount': None, 'price_currency_code':...",,http://blog.digg.com/?feed=rss2,http://blog.digg.com/,news,"[{'competitor': {'name': 'Reddit', 'permalink'...",Fri May 25 20:03:23 UTC 2007,http://www.crunchbase.com/company/digg,...,(415) 436-9638,"[{'name': 'Digg', 'permalink': 'digg'}]","[{'title': 'Public Relations', 'is_past': True...","[{'is_past': False, 'title': 'CEO', 'person': ...","[{'available_sizes': [[[117, 150], 'assets/ima...","community, social, news, bookmark, digg, techn...",$45M,digg,Tue Nov 05 21:35:47 UTC 2013,"[{'embed_code': '<embed src=""http://blip.tv/pl..."
1,52cdef7c4bab8bd675297d90,"{'price_amount': 625000000, 'price_currency_co...",[],None,,,web,"[{'competitor': {'name': 'RescueTime', 'permal...",Fri Jun 08 12:19:51 UTC 2007,http://www.crunchbase.com/company/postini,...,888.584.3150,"[{'name': 'Postini Solutions', 'permalink': 'p...",[],"[{'is_past': False, 'title': 'Board', 'person'...",[],,$0,None,Sat Aug 13 18:02:34 UTC 2011,[]
2,52cdef7c4bab8bd675297d92,"{'price_amount': 20000000, 'price_currency_cod...",[],None,http://www.flektor-blog.com/video_editing_soft...,http://www.flektor-blog.com,games_video,"[{'competitor': {'name': 'RockYou', 'permalink...",Thu May 31 21:11:51 UTC 2007,http://www.crunchbase.com/company/flektor,...,None,"[{'name': 'Flektor', 'permalink': 'flektor'}]",[],"[{'is_past': True, 'title': 'COO, Co-Founder',...",[],"flektor, photo, video",$0,None,Sat Nov 05 08:42:23 UTC 2011,[]
3,52cdef7c4bab8bd675297d91,"{'price_amount': None, 'price_currency_code': ...",[],,http://blog.geni.com/index.rdf,http://blog.geni.com,web,"[{'competitor': {'name': 'Ancestry', 'permalin...",Thu May 31 19:52:34 UTC 2007,http://www.crunchbase.com/company/geni,...,,"[{'name': 'Geni', 'permalink': 'geni'}]",[],"[{'is_past': False, 'title': 'CEO', 'person': ...",[],"geni, geneology, social, family, genealogy",$16.5M,geni,Wed Oct 10 14:01:29 UTC 2012,"[{'embed_code': '<object width=""425"" height=""3..."
4,52cdef7c4bab8bd675297d93,None,"[{'price_amount': 20000000, 'price_currency_co...",None,,,web,[],Thu May 31 21:46:57 UTC 2007,http://www.crunchbase.com/company/fox-interact...,...,,[],[],"[{'is_past': False, 'title': '', 'person': {'f...","[{'available_sizes': [[[150, 94], 'assets/imag...",,$0,twitterapi,Mon Aug 19 17:13:27 UTC 2013,"[{'embed_code': '<embed src=""http://blip.tv/pl..."


In [120]:
companies_with_coord['category_code'].value_counts()

web                 1931
software            1385
games_video          504
advertising          497
mobile               474
other                470
enterprise           414
network_hosting      334
consulting           334
ecommerce            331
public_relations     281
hardware             204
biotech              201
search               197
cleantech            169
security              85
semiconductor         65
analytics             44
social                37
finance               28
news                  23
education             22
medical               20
music                 19
travel                18
messaging             16
health                13
manufacturing         12
photo_video           11
legal                  9
real_estate            8
sports                 7
fashion                6
automotive             6
nanotech               5
hospitality            4
nonprofit              3
transportation         2
design                 2
local                  1


In [69]:
letters=[]
for i in companies_with_coord['total_money_raised']:
    if re.findall('[^0-9.,]+',i) not in letters:
        letters.append(re.findall('[^0-9.,]+',i))
    

In [70]:
letters

[['$', 'M'],
 ['$'],
 ['$', 'B'],
 ['$', 'k'],
 ['€', 'M'],
 ['£', 'k'],
 ['£', 'M'],
 ['C$', 'M'],
 ['€', 'k'],
 ['C$', 'k'],
 ['kr', 'M']]

In [71]:
dict_coin={
    '$' : 'USD',
    '£' : 'GBP',
    'C$' : 'CAD',
    'kr' : 'SEK',
    '€' : 'EUR'
}
coins= ['USD','GBP','CAD','SEK']

In [72]:
coindict={}
for i in coins:
    res = requests.get('https://api.exchangeratesapi.io/latest')
    data = res.json()
    coindict[i] = data['rates'][i]

In [73]:
coindict

{'USD': 1.1205, 'GBP': 0.89975, 'CAD': 1.4706, 'SEK': 10.6338}

In [74]:
dict_quantity={
    '$': 1,
    'M': 1000000, 
    'B':1000000000, 
    'k': 1000, 
    '£':0.89625/1.1215,  
    'C$':1.4642/1.1215, 
    'kr':10.5973/1.1215,
    '€':1/1.1215
}

In [75]:
dict_quantity['$']

1

In [76]:
total_money_raised_normalized=[]
for i in companies_with_coord['total_money_raised']:
    x = float(re.findall('[0-9.]+',str(i))[0])
    y=1
    for j in re.findall('[^0-9.]+',str(i)):
        y = y* dict_quantity[j]
    total_money_raised_normalized.append(x*y)

In [77]:
len(total_money_raised_normalized)

8406

In [78]:
companies_with_coord['total_money_raised_normalized_USD']= total_money_raised_normalized

In [79]:
for i in companies_with_coord['offices'].head():
    print(len(i))

1
1
1
1
1


In [80]:
companies_with_coord_intermediate=companies_with_coord.drop(columns='offices')

In [81]:
office_list=[]
for i in companies_with_coord['offices']:
    if not isinstance(i,list):
        office_list.append(list(i))
    else: office_list.append(i)

In [82]:
companies_with_coord['offices']=office_list

In [83]:
companies_with_coord2 = pd.DataFrame(companies_with_coord.offices.tolist()).stack().reset_index().drop(columns='level_1')
companies_with_coord2.columns=['index','office']

In [84]:
display(companies_with_coord2.head())

,index,office
0,0,"{'description': None, 'address1': '135 Mississ..."
1,1,"{'description': None, 'address1': '959 Skyway ..."
2,2,"{'description': None, 'address1': '8536 Nation..."
3,3,"{'description': 'Headquarters', 'address1': '9..."
4,4,"{'description': '', 'address1': '407 N Maple D..."


In [85]:
companies_with_coord.reset_index(inplace=True)

In [86]:
companies_with_coord.head()

,index,_id,acquisition,acquisitions,alias_list,blog_feed_url,blog_url,category_code,competitions,created_at,...,products,providerships,relationships,screenshots,tag_list,total_money_raised,twitter_username,updated_at,video_embeds,total_money_raised_normalized_USD
0,0,52cdef7c4bab8bd675297d8d,"{'price_amount': 500000, 'price_currency_code'...","[{'price_amount': None, 'price_currency_code':...",,http://blog.digg.com/?feed=rss2,http://blog.digg.com/,news,"[{'competitor': {'name': 'Reddit', 'permalink'...",Fri May 25 20:03:23 UTC 2007,...,"[{'name': 'Digg', 'permalink': 'digg'}]","[{'title': 'Public Relations', 'is_past': True...","[{'is_past': False, 'title': 'CEO', 'person': ...","[{'available_sizes': [[[117, 150], 'assets/ima...","community, social, news, bookmark, digg, techn...",$45M,digg,Tue Nov 05 21:35:47 UTC 2013,"[{'embed_code': '<embed src=""http://blip.tv/pl...",45000000.0
1,1,52cdef7c4bab8bd675297d90,"{'price_amount': 625000000, 'price_currency_co...",[],None,,,web,"[{'competitor': {'name': 'RescueTime', 'permal...",Fri Jun 08 12:19:51 UTC 2007,...,"[{'name': 'Postini Solutions', 'permalink': 'p...",[],"[{'is_past': False, 'title': 'Board', 'person'...",[],,$0,None,Sat Aug 13 18:02:34 UTC 2011,[],0.0
2,2,52cdef7c4bab8bd675297d92,"{'price_amount': 20000000, 'price_currency_cod...",[],None,http://www.flektor-blog.com/video_editing_soft...,http://www.flektor-blog.com,games_video,"[{'competitor': {'name': 'RockYou', 'permalink...",Thu May 31 21:11:51 UTC 2007,...,"[{'name': 'Flektor', 'permalink': 'flektor'}]",[],"[{'is_past': True, 'title': 'COO, Co-Founder',...",[],"flektor, photo, video",$0,None,Sat Nov 05 08:42:23 UTC 2011,[],0.0
3,3,52cdef7c4bab8bd675297d91,"{'price_amount': None, 'price_currency_code': ...",[],,http://blog.geni.com/index.rdf,http://blog.geni.com,web,"[{'competitor': {'name': 'Ancestry', 'permalin...",Thu May 31 19:52:34 UTC 2007,...,"[{'name': 'Geni', 'permalink': 'geni'}]",[],"[{'is_past': False, 'title': 'CEO', 'person': ...",[],"geni, geneology, social, family, genealogy",$16.5M,geni,Wed Oct 10 14:01:29 UTC 2012,"[{'embed_code': '<object width=""425"" height=""3...",16500000.0
4,4,52cdef7c4bab8bd675297d93,None,"[{'price_amount': 20000000, 'price_currency_co...",None,,,web,[],Thu May 31 21:46:57 UTC 2007,...,[],[],"[{'is_past': False, 'title': '', 'person': {'f...","[{'available_sizes': [[[150, 94], 'assets/imag...",,$0,twitterapi,Mon Aug 19 17:13:27 UTC 2013,"[{'embed_code': '<embed src=""http://blip.tv/pl...",0.0


In [87]:
companies_with_coord_offices= companies_with_coord2.merge(companies_with_coord, how='left', on='index')

In [88]:
companies_with_coord_offices.columns

Index(['index', 'office', '_id', 'acquisition', 'acquisitions', 'alias_list',
       'blog_feed_url', 'blog_url', 'category_code', 'competitions',
       'created_at', 'crunchbase_url', 'deadpooled_day', 'deadpooled_month',
       'deadpooled_url', 'deadpooled_year', 'description', 'email_address',
       'external_links', 'founded_day', 'founded_month', 'founded_year',
       'funding_rounds', 'homepage_url', 'image', 'investments', 'ipo',
       'milestones', 'name', 'number_of_employees', 'offices', 'overview',
       'partners', 'permalink', 'phone_number', 'products', 'providerships',
       'relationships', 'screenshots', 'tag_list', 'total_money_raised',
       'twitter_username', 'updated_at', 'video_embeds',
       'total_money_raised_normalized_USD'],
      dtype='object')

In [89]:
def getFirst(data):
    data = data['office']
#    return (len(data),data[0]['latitude'],data[0]['longitude'])

    # Only create the geoJSON object if all geodata is available
    principal = None
    if data['latitude'] and data['longitude']:
        principal = {
            "type":"Point",
            "coordinates":[data['longitude'], data['latitude']]
        }

    return {
        "lat": data['latitude'],
        "lng": data['longitude'],
        "oficina_principal": principal
    }

In [90]:
companies_with_coord_offices_sub = companies_with_coord_offices[["office"]].apply(getFirst, result_type="expand", axis=1)

In [91]:
companies_with_coord_offices_sub.head()

,lat,lng,oficina_principal
0,37.764726,-122.394523,"{'type': 'Point', 'coordinates': [-122.394523,..."
1,37.506885,-122.247573,"{'type': 'Point', 'coordinates': [-122.247573,..."
2,34.025958,-118.379768,"{'type': 'Point', 'coordinates': [-118.379768,..."
3,34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,..."
4,34.076179,-118.394170,"{'type': 'Point', 'coordinates': [-118.39417, ..."


In [92]:
companies_with_coord_offices_clean = pd.concat([companies_with_coord_offices,companies_with_coord_offices_sub], axis=1)

In [93]:
companies_with_coord_offices_clean.head()

,index,office,_id,acquisition,acquisitions,alias_list,blog_feed_url,blog_url,category_code,competitions,...,screenshots,tag_list,total_money_raised,twitter_username,updated_at,video_embeds,total_money_raised_normalized_USD,lat,lng,oficina_principal
0,0,"{'description': None, 'address1': '135 Mississ...",52cdef7c4bab8bd675297d8d,"{'price_amount': 500000, 'price_currency_code'...","[{'price_amount': None, 'price_currency_code':...",,http://blog.digg.com/?feed=rss2,http://blog.digg.com/,news,"[{'competitor': {'name': 'Reddit', 'permalink'...",...,"[{'available_sizes': [[[117, 150], 'assets/ima...","community, social, news, bookmark, digg, techn...",$45M,digg,Tue Nov 05 21:35:47 UTC 2013,"[{'embed_code': '<embed src=""http://blip.tv/pl...",45000000.0,37.764726,-122.394523,"{'type': 'Point', 'coordinates': [-122.394523,..."
1,1,"{'description': None, 'address1': '959 Skyway ...",52cdef7c4bab8bd675297d90,"{'price_amount': 625000000, 'price_currency_co...",[],None,,,web,"[{'competitor': {'name': 'RescueTime', 'permal...",...,[],,$0,None,Sat Aug 13 18:02:34 UTC 2011,[],0.0,37.506885,-122.247573,"{'type': 'Point', 'coordinates': [-122.247573,..."
2,2,"{'description': None, 'address1': '8536 Nation...",52cdef7c4bab8bd675297d92,"{'price_amount': 20000000, 'price_currency_cod...",[],None,http://www.flektor-blog.com/video_editing_soft...,http://www.flektor-blog.com,games_video,"[{'competitor': {'name': 'RockYou', 'permalink...",...,[],"flektor, photo, video",$0,None,Sat Nov 05 08:42:23 UTC 2011,[],0.0,34.025958,-118.379768,"{'type': 'Point', 'coordinates': [-118.379768,..."
3,3,"{'description': 'Headquarters', 'address1': '9...",52cdef7c4bab8bd675297d91,"{'price_amount': None, 'price_currency_code': ...",[],,http://blog.geni.com/index.rdf,http://blog.geni.com,web,"[{'competitor': {'name': 'Ancestry', 'permalin...",...,[],"geni, geneology, social, family, genealogy",$16.5M,geni,Wed Oct 10 14:01:29 UTC 2012,"[{'embed_code': '<object width=""425"" height=""3...",16500000.0,34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,..."
4,4,"{'description': '', 'address1': '407 N Maple D...",52cdef7c4bab8bd675297d93,None,"[{'price_amount': 20000000, 'price_currency_co...",None,,,web,[],...,"[{'available_sizes': [[[150, 94], 'assets/imag...",,$0,twitterapi,Mon Aug 19 17:13:27 UTC 2013,"[{'embed_code': '<embed src=""http://blip.tv/pl...",0.0,34.076179,-118.394170,"{'type': 'Point', 'coordinates': [-118.39417, ..."


In [95]:
companies_with_coord_offices_clean_final = companies_with_coord_offices_clean.drop(columns=['index','alias_list','blog_feed_url','blog_url','crunchbase_url','deadpooled_url','external_links','homepage_url','image','offices','permalink','twitter_username','video_embeds'])

In [96]:
companies_with_coord_offices_clean_final = companies_with_coord_offices_clean.drop(columns=['_id'])

In [97]:
companies_with_coord_offices_clean_final.head()

,index,office,acquisition,acquisitions,alias_list,blog_feed_url,blog_url,category_code,competitions,created_at,...,screenshots,tag_list,total_money_raised,twitter_username,updated_at,video_embeds,total_money_raised_normalized_USD,lat,lng,oficina_principal
0,0,"{'description': None, 'address1': '135 Mississ...","{'price_amount': 500000, 'price_currency_code'...","[{'price_amount': None, 'price_currency_code':...",,http://blog.digg.com/?feed=rss2,http://blog.digg.com/,news,"[{'competitor': {'name': 'Reddit', 'permalink'...",Fri May 25 20:03:23 UTC 2007,...,"[{'available_sizes': [[[117, 150], 'assets/ima...","community, social, news, bookmark, digg, techn...",$45M,digg,Tue Nov 05 21:35:47 UTC 2013,"[{'embed_code': '<embed src=""http://blip.tv/pl...",45000000.0,37.764726,-122.394523,"{'type': 'Point', 'coordinates': [-122.394523,..."
1,1,"{'description': None, 'address1': '959 Skyway ...","{'price_amount': 625000000, 'price_currency_co...",[],None,,,web,"[{'competitor': {'name': 'RescueTime', 'permal...",Fri Jun 08 12:19:51 UTC 2007,...,[],,$0,None,Sat Aug 13 18:02:34 UTC 2011,[],0.0,37.506885,-122.247573,"{'type': 'Point', 'coordinates': [-122.247573,..."
2,2,"{'description': None, 'address1': '8536 Nation...","{'price_amount': 20000000, 'price_currency_cod...",[],None,http://www.flektor-blog.com/video_editing_soft...,http://www.flektor-blog.com,games_video,"[{'competitor': {'name': 'RockYou', 'permalink...",Thu May 31 21:11:51 UTC 2007,...,[],"flektor, photo, video",$0,None,Sat Nov 05 08:42:23 UTC 2011,[],0.0,34.025958,-118.379768,"{'type': 'Point', 'coordinates': [-118.379768,..."
3,3,"{'description': 'Headquarters', 'address1': '9...","{'price_amount': None, 'price_currency_code': ...",[],,http://blog.geni.com/index.rdf,http://blog.geni.com,web,"[{'competitor': {'name': 'Ancestry', 'permalin...",Thu May 31 19:52:34 UTC 2007,...,[],"geni, geneology, social, family, genealogy",$16.5M,geni,Wed Oct 10 14:01:29 UTC 2012,"[{'embed_code': '<object width=""425"" height=""3...",16500000.0,34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,..."
4,4,"{'description': '', 'address1': '407 N Maple D...",None,"[{'price_amount': 20000000, 'price_currency_co...",None,,,web,[],Thu May 31 21:46:57 UTC 2007,...,"[{'available_sizes': [[[150, 94], 'assets/imag...",,$0,twitterapi,Mon Aug 19 17:13:27 UTC 2013,"[{'embed_code': '<embed src=""http://blip.tv/pl...",0.0,34.076179,-118.394170,"{'type': 'Point', 'coordinates': [-118.39417, ..."


In [99]:
companies_with_coord_offices_clean_final.shape

(9064, 47)

In [61]:
companies_with_coord_offices_clean_final.to_json('data_clean.json', orient="records")